<a href="https://colab.research.google.com/github/Yann-Xavier/Python/blob/main/Taxi_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
import numpy as np
import matplotlib.pyplot as plt
import os
import json

# Criar o ambiente
env = gym.make("Taxi-v3", render_mode="human")

# Ajuste de hiperparâmetros para melhorar o aprendizado
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.0003,
    n_steps=2048,
    batch_size=64,
    gamma=0.99,
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,  # Limitar o gradiente para estabilizar o aprendizado
    gae_lambda=0.95,    # Melhorar a estimativa de vantagens com GAE (Generalized Advantage Estimation)
    clip_range=0.2,     # Ajustar o intervalo de clipping para PPO
)

# Variáveis globais
total_points = 0
penalty_for_waiting = -0.1  # Penalidade por ficar parado
time_to_pick_penalty = 0.05  # Penalidade por demorar para pegar o passageiro
log_file_path = 'agent_scores.json'  # Caminho do arquivo de log


# Função para customizar recompensas
def custom_reward(action, reward, steps_without_pickup, has_passenger):
    global total_points

    # Penalidade por ação de 'esperar'
    if action == 4:
        reward += penalty_for_waiting

    # Penalidade adicional por tempo sem pegar o passageiro
    if not has_passenger:
        reward -= time_to_pick_penalty * steps_without_pickup

    # Recompensa adicional ao pegar ou entregar o passageiro
    if reward == 20:
        if not has_passenger:  # Pegou o passageiro
            total_points += 1
            print("Passageiro pego! +1 ponto")
        else:  # Deixou o passageiro
            total_points += 1
            print("Passageiro entregue ao hotel! +1 ponto")

    return reward


# Função para gerenciar a lógica de coleta e entrega de passageiros
def update_passenger_status(action, reward, has_passenger, steps_without_pickup):
    if reward == 20 and not has_passenger:  # Pegou o passageiro
        has_passenger = True
        steps_without_pickup = 0  # Resetar o contador de tempo
    elif reward == 20 and has_passenger:  # Deixou o passageiro
        has_passenger = False

    return has_passenger, steps_without_pickup


# Função para treinar o modelo
def train_model(model, total_timesteps):
    print("Iniciando o treinamento...")
    model.learn(total_timesteps=total_timesteps)


# Função para registrar a pontuação
def log_scores(score_history):
    # Salvar os scores no formato JSON
    if os.path.exists(log_file_path):
        with open(log_file_path, 'r') as f:
            scores_data = json.load(f)
    else:
        scores_data = []

    scores_data.append(score_history)

    with open(log_file_path, 'w') as f:
        json.dump(scores_data, f, indent=4)

    print(f"Pontuação salva em: {log_file_path}")


# Função para salvar o modelo treinado
def save_model(model, filename="taxi_agent_model"):
    model.save(filename)
    print(f"Modelo salvo como: {filename}.zip")


# Função para testar o modelo e coletar métricas de desempenho
def test_model(model, max_steps=1000):
    global total_points
    obs, info = env.reset()
    print("Executando o modelo treinado...")

    steps_without_pickup = 0
    has_passenger = False
    score_history = []  # Lista para armazenar a pontuação ao longo do tempo
    episode_rewards = []  # Recompensa total por episódio
    episode_lengths = []  # Número de passos por episódio

    for _ in range(max_steps):
        action, _ = model.predict(obs, deterministic=True)

        if isinstance(action, np.ndarray):
            action = action.item()

        # Observar o resultado da ação
        obs, reward, terminated, truncated, info = env.step(action)

        # Customizar a recompensa
        reward = custom_reward(action, reward, steps_without_pickup, has_passenger)

        # Atualizar o status do passageiro
        has_passenger, steps_without_pickup = update_passenger_status(action, reward, has_passenger, steps_without_pickup)

        # Penalidade com base no tempo
        if not has_passenger:
            steps_without_pickup += 1

        # Renderizar o ambiente
        env.render()

        # Atualizar a pontuação e salvar o histórico
        score_history.append(total_points)

        # Salvar as recompensas por episódio
        if terminated or truncated:
            episode_rewards.append(total_points)
            episode_lengths.append(_)
            print(f"Pontos acumulados neste episódio: {total_points}")
            obs, info = env.reset()
            total_points = 0
            steps_without_pickup = 0
            has_passenger = False

    # Plotar a pontuação durante a execução
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(score_history)
    plt.title("Pontuação ao longo do tempo")
    plt.xlabel("Passos")
    plt.ylabel("Pontuação")

    # Plotar recompensas e duração dos episódios
    plt.subplot(1, 2, 2)
    plt.plot(episode_rewards, label='Recompensa do Episódio')
    plt.plot(episode_lengths, label='Comprimento do Episódio')
    plt.title("Recompensas e Comprimento do Episódio")
    plt.xlabel("Episódios")
    plt.ylabel("Valor")
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Log de pontuação
    log_scores(score_history)

    env.close()
    print("Execução finalizada.")


# Executando o treinamento e teste
train_model(model, total_timesteps=500000)  # Aumentando os timesteps para maior exploração
test_model(model)

# Salvar o modelo treinado
save_model(model)


ModuleNotFoundError: No module named 'gymnasium'